In [7]:
import tushare as ts
import talib as ta
import os,time,sys,re,datetime
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import datetime as dt

In [8]:
df = ts.get_today_all()

In [9]:
df.head()

In [10]:
try:
    engine = create_engine('postgresql://' + 'postgres' + ':' + 'wangLee.' + '@localhost:5432/' + 'stock')
    c = engine.connect()
    conn = c.connection
except Exception as e:
    print('engine err!')

In [11]:
# 1)获得更新的所有标的数据
def getstock():
    df = ts.get_today_all()   #获取当日标的所有交易数据
    df = df[['code', 'name', 'open', 'high', 'trade', 'low', 'volume', 'amount']]
    df =df.set_index('code')
    df = df.sort_index()
    
    return df

# 2）、获取单只标的交易数据
def Get_Stock_Trade(code, conn):
    try:
        sql = "SELECT date,low,volume FROM _trade where code='" + code + "'"
        df = pd.read_sql(sql, conn)
        df = df.set_index('date')
        df = df.sort_index()

    except Exception as e:
        print(code, 'sql err!')

    return df


# 3)获取构造均价、均量、MACD数据
def Get_Stock_ma_vol(df_Code,date=None):
    print("starting.....")
    sql_list=[]
    no_arreay = []
    slen=df_Code.shape[0]
    i = 1

    for code,row in df_Code.iterrows():
        open,high,low,close,volume,amount = row[1:7]
        print(str(i)+"/"+str(slen)+","+code),
        i = i + 1

        try:
            df = Get_Stock_Trade(code, conn)  #获取标的之前的交易数据，以便于今日数据合并
            # your codes ，发现标的无记录，有的code在你的数据库表中无记录
             
                

        except Exception as e:
            print(code + 'trade err!')
            continue

        dflen = df.shape[0]

        #在历史交易df中加入今日的新记录
        if date is None:
            TODAY = dt.date.today()
            date=TODAY.strftime('%Y-%m-%d')
            
        new = pd.DataFrame('your codes')
        df=df.append(new)
        
        # your codes  构造成交价和成交量均线


      
        # your code 构造MACD


        
        sqls = []
        
        #print(code, date, open, high, low, close, volume, amount, macd, macdsignal, macdhist,ma5,ma25,vol5,vol60)
        sql = "insert into _trade(code,date,open,high,close,low,volume,amount,macd,macdsignal,macdhist,"\
        "ma5,ma25,vol5,vol60) values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%d','%.3f','%.3f',"\
"'%.3f','%.2f','%.2f','%d','%d')" % (code, date, open, high, low, close, volume, amount, macd, macdsignal, macdhist,ma5,ma25,vol5,vol60)
        
        sqls.append(sql)

        Insert_Stock_Trade(sqls, engine) ##追加数据库
        
    conn.close()
    print("End!")
    return no_arreay

def Insert_Stock_Trade(sqls, engine):
    try:
        for insert_sql in sqls:
            engine.execute(insert_sql)
            #print(insert_sql)
    except Exception as e:
        print('sql err!')

    return df


In [12]:
#调用各函数，完成每日数据更新
#your codes


### 任务列表
1、利用MACD数据判断底背离与顶背离，_trade表中最好包含所有标的的交易数据<br>
2、从_trade表中发现5日均量大于60日均量，25日均价线保持上升的标的<br>
3、利用 _trade 表数据制作透视表，使用数据框的pivot_table函数<br>
4、{一维Series，二维DataFrame，三维Panel，Panel中的每一项（类似于DataFrame的列）都是一个DataFrame}，利用 _trade表进行面板数据分析<br>
5、利用 _trade 表进行分组groupby（），交叉表crosstab的应用<br>
6、pandas 统计函数的应用<br>

count               非NA值的数量
describe            针对Series或各DataFrame列计算汇总统计
min,max             计算最小值和最大值
argmin,argmax       计算能够获取到最小值和最大值的索引位置（整数)
idxmin,idxmax       计算能够获取到最小值和最大值的索引值
quantile            计算样本的分位数（0到 1）
sum                 值的总和
mean                值的平均数， a.mean() 默认对每一列的数据求平均值；若加上参数a.mean(1)则对每一行求平均值
media               值的算术中位数（50%分位数)
mad                 根据平均值计算平均绝对离差
var                 样本值的方差
std                 样本值的标准差
skew                样本值的偏度（三阶矩）
kurt                样本值的峰度（四阶矩）
cumsum              样本值的累计和
cummin,cummax       样本值的累计最大值和累计最小
cumprod             样本值的累计积
diff                计算一阶差分（对时间序列很有用)
pct_change          计算百分数变化

### 透视表应用示例

In [13]:
#透视表  通过两张表_trade,_basic
date='2017-02-20'
sql = "SELECT A.date,A.code,A.low,A.volume,B.industry,B.area FROM _trade A,_basic B where A.code=B.code and A.date>='" + date + "'"
df = pd.read_sql(sql, conn)
df = df.set_index('date')
df = df.sort_index()

In [14]:
df.head()

In [15]:
tb=pd.pivot_table(df,values='volume',index=['industry'],columns=['area'], aggfunc=np.sum)

In [16]:
tb